In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [537]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from lifelines import CoxPHFitter

In [426]:
patient_df = pd.read_csv('data/allpatients_imputed_df').drop(columns=['Unnamed: 0','Unit1','Unit2'])

In [427]:
patient_df[patient_df.pid=="p00001"].head()
patient_df.shape
patient_df.HospAdmTime.unique().shape
patient_df[patient_df.SepsisLabel == 1].HospAdmTime.unique()

,pid,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,...,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,HospAdmTime,ICULOS,SepsisLabel
179972,p00001,93.0,92.5,36.5,110.0,76.0,56.0,22.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,1,0
179973,p00001,93.0,92.5,36.5,110.0,76.0,56.0,22.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,2,0
179974,p00001,91.0,96.0,36.5,108.0,84.5,72.0,23.5,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,3,0
179975,p00001,93.0,98.0,36.5,123.0,87.0,61.0,21.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,4,0
179976,p00001,93.0,95.0,36.5,110.0,81.0,70.0,20.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,5,0


(188453, 40)

(2976,)

array([ 0.00000e+00, -3.10000e-01, -5.39000e+01, -4.91000e+00,
       -5.00000e-02, -2.14600e+01, -1.32730e+02, -3.00470e+02,
       -1.92270e+02, -3.04000e+01, -8.01000e+00, -5.28000e+00,
       -2.43200e+01, -2.85800e+01, -4.91800e+01, -5.93740e+02,
       -2.81780e+02, -4.28000e+00, -3.53000e+01, -9.93000e+00,
       -1.20000e-01, -2.00000e-02, -5.63820e+02, -3.50260e+02,
       -3.19850e+02, -9.21000e+00, -6.03400e+01, -5.27240e+02,
       -2.47000e+00, -1.17172e+03, -7.52000e+00, -3.08740e+02,
       -1.15000e+00, -3.00000e-02, -2.60000e+00, -7.00000e-02,
       -2.52300e+01, -1.00000e-02, -7.76200e+01, -1.97780e+02,
       -7.08400e+01, -7.29000e+00, -2.58200e+01, -3.31000e+00,
       -2.66400e+01, -3.58800e+01, -9.75700e+01, -2.51100e+01,
       -2.92000e+00, -2.34000e+00, -1.01100e+01, -1.10000e-01,
       -1.19900e+01, -4.40000e-01, -9.07000e+00, -1.70000e-01,
       -2.69600e+01, -3.64940e+02, -7.21200e+01, -4.95000e+00,
       -2.91000e+00, -4.00000e-02, -8.00000e-02, -3.300

In [130]:
#patient_df = patient_df.fillna(patient_df.mean()['HR':'Platelets'])

In [428]:
patient_df[patient_df.pid=="p00001"].SepsisLabel.unique()

array([0], dtype=int64)

In [429]:
patient_df.pid.unique().shape

(5000,)

In [437]:
re_idx = patient_df.set_index(["pid","ICULOS"])
arr = re_idx.groupby("pid").apply(lambda x: x.values).values

train_raw, test_raw = train_test_split(arr, test_size=0.2)

In [438]:
train_raw[0].shape
train_raw[0][0]

(14, 38)

array([  74.5       ,  100.        ,   36.45      ,  110.5       ,
         84.        ,   69.5       ,   16.        ,   18.5       ,
         -3.16091766,   22.72753865,    0.8       ,    7.4       ,
         40.        ,   96.5030266 ,  116.31360923,    8.        ,
         91.34764342,    1.11      ,  105.        ,    0.86      ,
          0.85336036,  131.        ,    1.47      ,    2.3       ,
          4.1       ,    4.2       ,    1.44430028,    5.67129436,
         25.7       ,    8.        ,   40.80889022,   14.3       ,
        296.93274106,  316.        ,   50.        ,    1.        ,
       -102.04      ,    0.        ])

In [463]:
# create windowing system here
T = 6
#idx = 10
def process_data(df: pd.DataFrame, T: int) -> (pd.DataFrame, np.array):
    processed = []
    labels = []

    for idx in range(df.shape[0]):
        if df[idx][:,-1].sum() == 0:
            processed.extend([[row,7,1] for row in df[idx][:,:-1]])
        else:
            sepsis_count = 0
            for i in range(df[idx].shape[0]):
                t = (T + 1) - sepsis_count
                t = t if t >= 1 else 1
                s = 1 if t > T else 0
                processed.append([df[idx][i][:-1],t,s])
                sepsis_count += 1 if df[idx][i][-1] == 1 else 0
                
        labels.extend(df[idx][:,-1].tolist())
                
    return (pd.DataFrame(processed, columns=["x","t","s"]), np.array(labels))
# Naive windowing:
#             for i in range(df[idx].shape[0]):
#                 window = df[idx][i:i+T]
#                 matches = np.where(window[:,-1]==1)[0]
#                 if matches.size > 0:
#                     t = matches[0] + 1
#                     s = 0
#                 else:
#                     t = T + 1
#                     s = 1
#                 processed.append([df[idx][i][:-1],t,s])

# def test(df: pd.DataFrame, T: int) -> pd.DataFrame:
#     processed = []

#     #for idx in range(df.shape[0]):
#     idx = 10
#     if df[idx][:,-1].sum() == 0:
#         processed.extend([[row,7,1] for row in df[idx][:,:-1]])
#     else:
#         sepsis_count = 0
#         for i in range(df[idx].shape[0]):
#             t = (T + 1) - sepsis_count
#             t = t if t >= 1 else 1
#             s = 1 if t > T else 0
#             processed.append([df[idx][i][:-1],t,s])
#             sepsis_count += 1 if df[idx][i][-1] == 1 else 0
                
#     return pd.DataFrame(processed, columns=["x","t","s"])

# np.array(train[idx][:,:-1]).shape
# np.repeat([[7,1]],train[idx].shape[0],axis=0).shape
# t_s = np.repeat([[7,1]],train[idx].shape[0],axis=0)
# t_s
# x_idx = train[idx][:,:-1].reshape(train[idx][:,:-1].shape[0],1,-1)
# x_idx.shape
# processed_idx = np.concatenate((x_idx,t_s),axis=1)
# processed_idx.shape
#if train[idx][:,-1].sum() == 0:
# a = np.ones((4,1)) * 3
# b = np.repeat([[7,1]],4,axis=0)    
# a
# b
# np.hstack((a,b))
#pd.DataFrame([[train[idx][i][:-1],t,s]], columns=["x","t","s"])
# for idx in range(4000):
#     #window = train[idx][i:i+T]
#     if train[idx][:,-1].sum() > 0:
#         break

In [464]:
X_train, y_train = process_data(train_raw, T)
X_test, y_test = process_data(test_raw, T)

In [505]:
inverse_s = 1-X_train.s
X_train_cph = pd.DataFrame(X_train.x.values.tolist(), columns=patient_df.columns[1:-2])
X_train_cph["s"] = inverse_s
X_train_cph["t"] = X_train.t

In [524]:
cph = CoxPHFitter(penalizer=0.0)
cph.fit(X_train_cph, duration_col='t', event_col='s', step_size=0.075, show_progress=True)

Iteration 1: norm_delta = 0.07524, step_size = 0.0750, ll = -22136.64393, newton_decrement = 815.98703, seconds_since_start = 0.1
Iteration 2: norm_delta = 0.06459, step_size = 0.0750, ll = -22019.03646, newton_decrement = 619.72318, seconds_since_start = 0.2
Iteration 3: norm_delta = 0.05742, step_size = 0.0750, ll = -21929.65184, newton_decrement = 491.48916, seconds_since_start = 0.4
Iteration 4: norm_delta = 0.06216, step_size = 0.0900, ll = -21858.74541, newton_decrement = 396.61441, seconds_since_start = 0.5
Iteration 5: norm_delta = 0.05450, step_size = 0.0882, ll = -21790.62665, newton_decrement = 309.89275, seconds_since_start = 0.7
Iteration 6: norm_delta = 0.04843, step_size = 0.0864, ll = -21738.40682, newton_decrement = 246.22308, seconds_since_start = 0.8
Iteration 7: norm_delta = 0.05321, step_size = 0.1037, ll = -21697.70221, newton_decrement = 198.23021, seconds_since_start = 1.0
Iteration 8: norm_delta = 0.04717, step_size = 0.1016, ll = -21658.73669, newton_decrement

<lifelines.CoxPHFitter: fitted with 151268 observations, 149411 censored>

In [527]:
#cph.check_assumptions(X_train_cph,show_plots=False,plot_n_bootstraps=0)
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 151268 observations, 149411 censored>
      duration col = 't'
         event col = 's'
number of subjects = 151268
  number of events = 1857
    log-likelihood = -21514.46
  time fit was run = 2019-03-19 22:20:15 UTC

---
                  coef  exp(coef)  se(coef)      z      p  -log2(p)  lower 0.95  upper 0.95
HR                0.01       1.01      0.00   9.85 <0.005     73.68        0.01        0.02
O2Sat            -0.01       0.99      0.01  -1.81   0.07      3.85       -0.02        0.00
Temp              0.34       1.40      0.03  10.11 <0.005     77.37        0.27        0.40
SBP              -0.00       1.00      0.00  -0.59   0.56      0.84       -0.00        0.00
MAP               0.00       1.00      0.00   0.02   0.98      0.02       -0.01        0.01
DBP              -0.01       0.99      0.00  -1.58   0.11      3.13       -0.01        0.00
Resp              0.01       1.01      0.00   1.49   0.14      2.88       -0.00        0.01
EtCO2

In [515]:
train_preds = 1-cph.predict_survival_function(X_train_cph.drop(columns=["s","t"]),times=[6])

In [522]:
X_train_cph[0:30]
train_preds[0:30]
y_train[0:30]

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,HospAdmTime,s,t
0,74.5,100.0,36.45,110.5,84.0,69.5,16.0,18.500000,-3.160918,22.727539,...,8.000000,40.80889,14.300000,296.932741,316.000000,50.0,1.0,-102.04,0,7
1,74.5,100.0,36.45,110.5,84.0,69.5,16.0,18.500000,-3.160918,22.727539,...,8.000000,40.80889,14.300000,296.932741,316.000000,50.0,1.0,-102.04,0,7
2,81.0,97.0,36.80,113.5,88.5,74.0,18.0,18.500000,-3.160918,22.727539,...,8.000000,40.80889,14.300000,296.932741,316.000000,50.0,1.0,-102.04,0,7
3,69.0,100.0,36.70,103.0,79.0,66.0,24.0,18.500000,-3.160918,22.727539,...,8.000000,40.80889,14.300000,296.932741,316.000000,50.0,1.0,-102.04,0,7
4,67.0,100.0,36.70,100.0,77.0,65.0,24.0,18.500000,-3.160918,22.727539,...,8.000000,40.80889,14.300000,296.932741,316.000000,50.0,1.0,-102.04,0,7
5,62.0,100.0,36.50,109.0,82.0,67.0,22.0,18.500000,-3.160918,22.727539,...,8.000000,40.80889,14.300000,296.932741,316.000000,50.0,1.0,-102.04,0,7
6,60.0,100.0,36.70,91.0,69.0,57.0,22.0,18.500000,-3.160918,22.727539,...,8.000000,40.80889,14.300000,296.932741,316.000000,50.0,1.0,-102.04,0,7
7,64.0,96.0,36.80,89.0,66.0,56.0,22.0,18.500000,-3.160918,22.727539,...,8.000000,40.80889,14.300000,296.932741,316.000000,50.0,1.0,-102.04,0,7
8,63.0,98.0,36.60,86.0,62.0,52.0,22.0,18.500000,-3.160918,22.727539,...,8.000000,40.80889,14.300000,296.932741,316.000000,50.0,1.0,-102.04,0,7
9,81.0,95.0,36.50,116.0,73.0,57.0,22.0,18.500000,-3.160918,22.727539,...,8.000000,40.80889,14.300000,296.932741,316.000000,50.0,1.0,-102.04,0,7


0     0.017715
1     0.017715
2     0.021801
3     0.019130
4     0.018768
5     0.015888
6     0.008385
7     0.009598
8     0.008845
9     0.010613
10    0.010231
11    0.010105
12    0.008773
13    0.008617
14    0.008420
15    0.008420
16    0.008725
17    0.008253
18    0.008832
19    0.008712
20    0.008110
21    0.008110
22    0.008110
23    0.008110
24    0.008110
25    0.008110
26    0.004350
27    0.004350
28    0.003957
29    0.003870
Name: 6, dtype: float64

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.])

In [538]:
lst = [1 if i >=0.5 else 0 for i in train_preds]
((lst == y_train).sum() / y_train.shape[0]) * 100
roc_auc_score(y_train, train_preds)

98.62099055980114

0.7081481949914846

In [445]:
# test simple cphfitter from lifelines

# implement actual cox-weibull from "scratch"?